# Автобрея. ДЗ2

Дорогая,  Екатерина Владимировна! 
Просто хочу напомнить Вам, что мы с вами договорились о том, что я сдам дз посед дэдлайна без штрафа, так как у меня была официальная "отсрочка".

**Часть 1. Отфильтровываем словосочетания**

In [30]:
from pymorphy2 import MorphAnalyzer
from nltk.tokenize import RegexpTokenizer
from collections import Counter
from nltk.parse import DependencyGraph

In [31]:
!C:\Users\asus\Downloads\udpipe-1.2.0-bin\udpipe-1.2.0-bin\bin-win64\udpipe --input horizontal --output conllu \
--tokenize --tag --parse \
C:\Users\asus\Documents\doc\russian-syntagrus-ud-2.4-190531.udpipe \
< text.txt > text.conllu

Loading UDPipe model: done.


In [32]:
with open('text.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [85]:
trees = []

with open('text.conllu', 'r', encoding = 'utf-8') as f:
    parsed_sents = f.read().split('\n\n')

    for sent in parsed_sents:
        tree = [line for line in sent.split('\n') if line and line[0] != '#']
        trees.append('\n'.join(tree))

In [86]:
m = MorphAnalyzer()
tokenizer = RegexpTokenizer(r'\w+')

In [87]:
def normalize(text):
    tokens = tokenizer.tokenize(text.lower())
    lemmas = [m.parse(token)[0].normal_form for token in tokens]
    return lemmas #возвращает массив

In [88]:
lemmas = normalize(text)
lemmas_counter = Counter(lemmas)
over_50 = [x[0] for x in lemmas_counter.items() if x[1] >= 50]

In [89]:
over_50[:20]

['я',
 'бы',
 'единый',
 'россия',
 'в',
 'один',
 'не',
 'только',
 'депутат',
 'но',
 'и',
 'суд',
 'прокуратура',
 'заявить',
 'ъ',
 'член',
 'борис',
 'тогда',
 'о',
 'власть']

In [90]:
collocations = []
for i, tree in enumerate(trees):
    try:
        g = DependencyGraph(tree, top_relation_label='root')
        
        for line in g.triples():
            # (('предложил', 'VERB'), 'punct', ('"', 'PUNCT'))
            if line[0][1] == 'VERB' and line[1] == 'obj' and line[2][1] == 'NOUN' and len(line[0][0]) > 3:
                # проверка длины поможет избавиться от мусора, так как в глаголы почему-то попали 
                # также "в", "и" и м б некоторые другие служебные слова, которые набирают очень 
                # высокие скоры словосочетаний и всё портят                
                verb = normalize(line[0][0])
                if verb[0] in over_50:
                    collocations.append((verb[0], normalize(line[2][0])[0]))
                    
    except:
        pass

In [91]:
collocations[:10]

[('подать', 'иск'),
 ('признать', 'договор'),
 ('рассмотреть', 'вопрос'),
 ('обвинить', 'руководство'),
 ('получить', 'срок'),
 ('обвинить', 'олигарх'),
 ('направить', 'ведомство'),
 ('просить', 'суд'),
 ('сделать', 'заявление'),
 ('рассматривать', 'ходатайство')]

**Часть 2 и 3. Метрики**

In [92]:
import nltk
from nltk.collocations import *

In [93]:
bigram_measures = nltk.collocations.BigramAssocMeasures()
finder = BigramCollocationFinder.from_documents(collocations)

Вот тут будут лежать скоры. Мы же сразу перейдём к топу 100 и его пересечению

In [94]:
likelihood_scores = finder.score_ngrams(bigram_measures.likelihood_ratio)
dice_scores = finder.score_ngrams(bigram_measures.dice)
pmi_scores = finder.score_ngrams(bigram_measures.pmi)

In [95]:
likelihood_scores[:10]

[(('подать', 'иск'), 380.19748667174326),
 (('представлять', 'интерес'), 276.8897944059793),
 (('выдать', 'санкция'), 228.65957556686018),
 (('принять', 'решение'), 204.95762712617565),
 (('удовлетворить', 'иск'), 179.08076220192416),
 (('вынести', 'приговор'), 176.86441368440072),
 (('попросить', 'суд'), 174.25792107286),
 (('обжаловать', 'решение'), 167.46986225687957),
 (('просить', 'суд'), 149.06891601473905),
 (('вынести', 'решение'), 125.19970185458662)]

In [96]:
dice_scores [:10]

[(('газета', 'предупреждение'), 1.0),
 (('начало', 'разбирательство'), 1.0),
 (('рассказать', 'замгендиректор'), 1.0),
 (('являться', 'долг'), 1.0),
 (('представлять', 'интерес'), 0.84375),
 (('выдать', 'санкция'), 0.75),
 (('есть', 'признак'), 0.6666666666666666),
 (('напомнить', 'основание'), 0.6666666666666666),
 (('отказаться', 'падение'), 0.6666666666666666),
 (('стать', 'сальмонелла'), 0.6666666666666666)]

In [97]:
pmi_scores [:10]

[(('газета', 'предупреждение'), 12.026523442519766),
 (('начало', 'разбирательство'), 12.026523442519766),
 (('рассказать', 'замгендиректор'), 12.026523442519766),
 (('являться', 'долг'), 12.026523442519766),
 (('есть', 'признак'), 11.026523442519766),
 (('отказаться', 'падение'), 11.026523442519766),
 (('стать', 'сальмонелла'), 11.026523442519766),
 (('обратиться', 'ответ'), 10.44156094179861),
 (('обратиться', 'родные'), 10.44156094179861),
 (('пояснить', 'причина'), 10.44156094179861)]

 а теперь перейдём к нахождению топов и пересечению

In [98]:
likelihood = finder.nbest(bigram_measures.likelihood_ratio, 100)
likelihood[:10]

[('подать', 'иск'),
 ('представлять', 'интерес'),
 ('выдать', 'санкция'),
 ('принять', 'решение'),
 ('удовлетворить', 'иск'),
 ('вынести', 'приговор'),
 ('попросить', 'суд'),
 ('обжаловать', 'решение'),
 ('просить', 'суд'),
 ('вынести', 'решение')]

In [99]:
pmi = finder.nbest(bigram_measures.pmi, 100)
pmi[:10]

[('газета', 'предупреждение'),
 ('начало', 'разбирательство'),
 ('рассказать', 'замгендиректор'),
 ('являться', 'долг'),
 ('есть', 'признак'),
 ('отказаться', 'падение'),
 ('стать', 'сальмонелла'),
 ('обратиться', 'ответ'),
 ('обратиться', 'родные'),
 ('пояснить', 'причина')]

In [100]:
dice = finder.nbest(bigram_measures.dice, 100)
dice[:10]

[('газета', 'предупреждение'),
 ('начало', 'разбирательство'),
 ('рассказать', 'замгендиректор'),
 ('являться', 'долг'),
 ('представлять', 'интерес'),
 ('выдать', 'санкция'),
 ('есть', 'признак'),
 ('напомнить', 'основание'),
 ('отказаться', 'падение'),
 ('стать', 'сальмонелла')]

In [101]:
intersection = set(likelihood) & set(dice) & set(pmi)
intersection

{('арестовать', 'имущество'),
 ('взыскать', 'задолженность'),
 ('взыскать', 'неустойка'),
 ('доказать', 'незаконность'),
 ('запретить', 'деятельность'),
 ('заявить', 'отвод'),
 ('назначить', 'наказание'),
 ('напомнить', 'основание'),
 ('объявить', 'голодовка'),
 ('объявить', 'перерыв'),
 ('объявить', 'предприниматель'),
 ('отказать', 'оао'),
 ('решить', 'проблема')}

Ну, скажем так, это не идеальный результат, но всё же...
Объявить предприниматель - это ошибка изначальной синтаксической разметки. Там "обвинение - предприниматель", которое странно расспарсилось и, возможно, из-за специфики текста попало в топ. Также из-за этого туда попало " отказать оао"

In [102]:
with open('verb_coll.txt', 'r', encoding='utf-8') as f:
    lines = f.readlines()
    
verb_dict = [tuple(normalize(line.split('\t')[-1].strip())) for line in lines]

In [103]:
verb_dict[:10]

[('аборт', 'делать'),
 ('делать', 'аборт'),
 ('сделать', 'аборт'),
 ('авария', 'произойти'),
 ('авария', 'случиться'),
 ('потерпеть', 'авария'),
 ('произойти', 'авария'),
 ('случиться', 'авария'),
 ('завоевать', 'авторитет'),
 ('подорвать', 'авторитет')]

In [104]:
golden_standart = intersection & set(verb_dict)

In [105]:
golden_standart

{('объявить', 'перерыв'), ('решить', 'проблема')}

И на этом этапе я начинаю понимать, что нам, правда, надо перейти к этапу 4, где мы добавляем всё ручками))

**Часть 4. NLP - это руками (с)**

так как имеющихся двух словосочетаний в золотом стандарте, конечно же, недостаточно, добавим еще выражений из топов

При добавлении следуюших словосочетаний я руковдствовался тем, что глагол и сушествительное  ав них (согласно моей интуиции носителя) встречаются в месте чаще, чем стоило бы ожидать в среднем ('вынести приговор'), что глагол или существительное в этом словосочетании было бы странно заменить синонимом (т. е. это, может быть, даже и было бы грамматично,но так, как правило, не делают: принять решение - взять решение, дать показания - предоставить/направить показания) и тем, что порою эти слова вместе имеют некомпозициональное значение ('принять решение', 'представлять интерес')- т. е. не могут быть выведены как "сумма" прямых значений составляющих их слов

In [106]:
to_add = {('подать', 'иск'), ('представлять', 'интерес'), ('принять', 'решение'), ('вынести', 'приговор'), ('сделать', 'заявление'), ('дать', 'показание'), ('дать', 'оценка')}

In [107]:
 final_set =  golden_standart| to_add

In [108]:
final_set

{('вынести', 'приговор'),
 ('дать', 'оценка'),
 ('дать', 'показание'),
 ('объявить', 'перерыв'),
 ('подать', 'иск'),
 ('представлять', 'интерес'),
 ('принять', 'решение'),
 ('решить', 'проблема'),
 ('сделать', 'заявление')}

**Часть 5. Оценка качества**

In [117]:
from scipy.stats import spearmanr

Для работы со Спирманном нам нужно как-то отранжировать полученные биграммы. Своей интуицией я отранжирую их так.

In [110]:
ranged = [
    ('представлять', 'интерес'),
    ('принять', 'решение'),
    ('дать', 'оценка'),
    ('дать', 'показание'),
    ('вынести', 'приговор'),
    ('сделать', 'заявление'),
    ('подать', 'иск'),
    ('решить', 'проблема'),
    ('объявить', 'перерыв')
]

Если мы просто прогоним по метрике тьюплы, результаты получатся не читабельными

In [112]:
spearmanr(pmi[:len(ranged)],ranged)

SpearmanrResult(correlation=array([[ 1.        , -0.30962614, -0.13445378, -0.1924703 ],
       [-0.30962614,  1.        ,  0.07531447, -0.01666667],
       [-0.13445378,  0.07531447,  1.        , -0.16736548],
       [-0.1924703 , -0.01666667, -0.16736548,  1.        ]]), pvalue=array([[0.        , 0.41747792, 0.73018588, 0.61980614],
       [0.41747792, 0.        , 0.84729402, 0.9660548 ],
       [0.73018588, 0.84729402, 0.        , 0.66690464],
       [0.61980614, 0.9660548 , 0.66690464, 0.        ]]))

Поэтому

In [113]:
def stringing(ranged):
    res = []
    for item in ranged:
        res.append(' '.join(item))
    return res 

In [116]:
pmi = stringing(pmi)
dice = stringing(dice)
likelihood = stringing(likelihood)
ranged = stringing(ranged)

In [120]:
spearmanr(likelihood[:len(ranged)],ranged)

SpearmanrResult(correlation=-0.4666666666666666, pvalue=0.20538635110581185)

In [121]:
spearmanr(dice[:len(ranged)],ranged)

SpearmanrResult(correlation=-0.7999999999999999, pvalue=0.00962792472537983)

In [122]:
spearmanr(pmi[:len(ranged)],ranged)

SpearmanrResult(correlation=-0.15, pvalue=0.7000942309691419)

По метрикам получается, что dice ведёт себя лучше всех,но мне субъективно больше понравились результаты likelihood